In [1]:
from __future__ import print_function
import torch
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

In [2]:
class CurrencyDataset(Dataset):
    def __init__(self,root_dir):
        self.dir = os.path.abspath(root_dir)
    def __getitem__(self,idx):
        data_path = os.path.join(self.dir,'Batches','Batch_' + str(idx) + '.npy')
        label_path = os.path.join(self.dir,'Labels','Label_' + str(idx) + '.npy')
        data = np.load(data_path)
        labels = np.load(label_path)
        return (data,labels)
    def __len__(self):
        return len(os.listdir(os.path.join(self.dir,'Batches')))

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(2, 50, (3,1))
        self.conv2 = nn.Conv2d(50, 16, (48,1))
        self.fc1 = nn.Linear(48, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 48)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = F.softmax(x,dim=1)
        return x

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor')
data = CurrencyDataset('./Processed')
net = Net()
net = net.double()
net = net.cuda()
dataloader = DataLoader(data,batch_size=10,shuffle=True,pin_memory=True)

In [4]:
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [5]:
running_loss = 0.0
for i,batch in enumerate(dataloader):
    optimizer.zero_grad()
    d,y = batch
    d = d.cuda()
    x = net(d)
    y = (y.reshape([-1,2])).cuda()
    z = torch.mul(x,y)
    z = z.sum(dim=1)
    loss = z.sum()
    loss /= z.shape[0]
    loss = loss ** -1
    print(loss)
    loss.backward()
    optimizer.step()

tensor(1.0014, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0007, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0015, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0008, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0011, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0016, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0008, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0006, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0014, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0012, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0007, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0029, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0020, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0016, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0014, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0006, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0009, device='cuda:0', grad_fn=<PowBackward0>)
tensor(1.0007, device='cuda:0', grad_fn=<PowBack

In [12]:
for i,batch in enumerate(dataloader):
    d,y = batch
    print(net(d.cuda()))
    break

tensor([[3.7119e-03, 9.9629e-01],
        [2.6111e-04, 9.9974e-01],
        [3.9116e-03, 9.9609e-01],
        [3.7642e-03, 9.9624e-01],
        [2.6820e-03, 9.9732e-01],
        [3.8766e-03, 9.9612e-01],
        [3.7328e-03, 9.9627e-01],
        [3.6195e-03, 9.9638e-01],
        [3.8344e-03, 9.9617e-01],
        [3.8095e-03, 9.9619e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
